In [2]:
%run train_model.py

/home/emac1995/devops_course/task2/Deploying-a-Scalable-ML-Pipeline-with-FastAPI/fastapi/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Precision: 0.7164 | Recall: 0.2458 | F1: 0.3661


In [5]:
%run train_model_test_code.py

/home/emac1995/devops_course/task2/Deploying-a-Scalable-ML-Pipeline-with-FastAPI/fastapi/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Precision: 0.7285 | Recall: 0.2699 | F1: 0.3939


In [1]:
%run test_ml.py

ModuleNotFoundError: No module named 'pytest'

In [ ]:
#autopep8 --in-place --aggressive --aggressive test_ml.py

In [4]:
%run main.py